In [1]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *
DATA_DIR = '/work/users/k/4/k4thryn/Repos/OLD_EpSampling_Nov2024/data/'

pd.set_option('display.max_columns', None)

In [2]:
from epsampling.utils import get_chunks
from epsampling.modeling import get_date_chunked_splits, get_performance
from sklearn import linear_model
from sklearn.model_selection import train_test_split

def get_model_df(df, X_COLS, Y_COL, NAIVE_COL):
    cols = ['Date', 'Fips'] + X_COLS + [Y_COL, NAIVE_COL]
    df = df[cols]    
    return df

def get_df_res(df_train, df_test, X_COLS, Y_COL, PRED_COL, ALG):    
    X_train = df_train[X_COLS]
    y_train = df_train[Y_COL]
    X_test = df_test[X_COLS]
    
    df_pred = df_test.copy()
    reg = linear_model.LinearRegression().fit(X_train, y_train)
    df_pred['Algorithm'] = ALG
    df_pred[PRED_COL] = reg.predict(X_test)
    
    return df_pred

In [8]:
# problem dfs:
# XX_blockgroup_acs5_2021_healthinsurance.csv
# XX_blockgroup_acs5_2021_income.csv

from epsampling.utils import drop_sers_with_nans
from epsampling.utils import date_str_to_int

def normalize_attrs_by_pop(df, f):
    f = (f[f.rindex('/')+1:-4])
    f = f[f.rindex('_')+1:]
    
    df.set_index('GEOID',drop=True,inplace=True)

    if f=='healthinsurance':
        df['HINS_A0018'] = df['HINS_A0018'].div(df['POP_A0018'])
        df['HINS_A1934'] = df['HINS_A1934'].div(df['POP_A1934'])
        df['HINS_A3564'] = df['HINS_A3564'].div(df['POP_A3564'])
        df['HINS_A65p'] = df['HINS_A65p'].div(df['POP_A65p'])
        
        dff = df.drop(['POP_A0018','POP_A1934','POP_A1934','POP_A65p'],inplace=False,axis=1)
        
    elif f=='income':
        denom = df['HH']
        
        dff = df.apply(lambda x: x/denom, axis=0) 
        ## fix MHI since its not supposed to be normalized
        dff['MHI'] = df['MHI']
        dff['HH'] = denom

    else:
        universe = df.columns[0]
        denom = df[universe]
        
        dff = df.apply(lambda x: x/denom, axis=0)
        dff[universe] = denom
        
    dff = dff.reset_index(inplace=False, drop=False)
    return dff

def get_state_df(files, normalize=False):
    first_df = None
    for i,f in enumerate(files):
        this_df = pd.read_csv(f)
        if normalize:
            this_df = normalize_attrs_by_pop(this_df, f)
        if first_df is None:
            first_df = this_df
        else:
            df = pd.merge(first_df, this_df, on='GEOID', suffixes=(f'_x{i}', f'_x{i+1}'))
            first_df = df
    return df


import glob
import os


def get_acs_df():
    
    acs_dir = '/work/users/k/4/k4thryn/Repos/EpSampling/data/raw/acs_results/'

    all_st_dfs = []

    state_dirs = [x for x in os.walk(acs_dir)][0][1]

    # print(state_dirs)
    state_dirs = ['NC']

    for i,state in enumerate(state_dirs):
    #     if i==0:
    #         continue    

        files = glob.glob(f'{acs_dir}{state}/*.csv')
        df = get_state_df(files)
        ## REFORMAT dataframe ... rename cols.
        df.rename({'GEOID':'Fips'},axis=1,inplace=True)
        all_st_dfs.append(df)

    df = pd.concat(all_st_dfs)
    df.reset_index(drop=True,inplace=True)

    ## check for cols with nans.
    df = drop_sers_with_nans(df, from_axis='cols')

    ## reset index
    df.reset_index(inplace=True,drop=True)

    df.set_index('Fips',drop=True,inplace=True)

    ## SAVE CSV!
    # fpath = os.path.join(DATA_DIR,'processed','scratch',f'formatted_acs_results_{DT}.csv')
    # fpath = os.path.join(DATA_DIR,'processed',f'formatted_acs_results_normed_{DT}.csv')
    # df.to_csv(fpath,index=False)
    # print(DT)

    ## READ BACK AND CHECK
    # dff = pd.read_csv(fpath)
    # display(df, dff)

    return df

In [11]:
df = get_acs_df()

ts = '20241108-123515'
fpath = os.path.join(DATA_DIR,'processed',f'training_target_df_{ts}.csv')
dff = pd.read_csv(fpath)
df = pd.merge(dff, df, on='Fips')

display(df)

,Date,State_fips,Fips,Pop,State_pop,Ratio,Proj_state_inc_deaths,True_state_inc_deaths,True_county_cum_deaths,True_county_inc_deaths,Naive_proj_deaths,Naive_true_deaths,True_county_inc_deaths_tm_4,True_county_inc_deaths_tm_5,True_county_inc_deaths_tm_6,True_county_inc_deaths_tm_7,True_county_inc_deaths_tm_8,True_county_inc_deaths_tm_9,True_county_inc_deaths_tm_10,True_county_inc_deaths_tm_11,POP_x2,POP_M,POP_F,POP_A0004,POP_A0509,POP_A1014,POP_A1517,POP_A1819,POP_A20,POP_A21,POP_A2224,POP_A2529,POP_A3034,POP_A3539,POP_A4044,POP_A4549,POP_A5054,POP_A5559,POP_A6061,POP_A6264,POP_A6566,POP_A6769,POP_A7074,POP_A7579,POP_A8084,POP_A85p,POP_A25p,EDU_ltHS,EDU_HS,EDU_ASSOC,EDU_BACH,EDU_MAST,EDU_PROF,EDU_DOCT,POP_x3,POP_NH,POP_HISP_x12,POP_A0018,POP_A1934,POP_A3564,POP_A65p,HINS_A0018,HINS_A1934,HINS_A3564,HINS_A65p,HH_x5,HH_1P,HH_2P,HH_3P,HH_4P,HH_5P,HH_6P,HH_7pP,MHI,HH_x6,HHI_00_10k,HHI_10_15k,HHI_15_20k,HHI_20_25k,HHI_25_30k,HHI_30_35k,HHI_35_40k,HHI_40_45k,HHI_45_50k,HHI_50_60k,HHI_60_75k,HHI_75_100k,HHI_100_125k,HHI_125_150k,HHI_150_200k,HHI_200k_p,POP_16p_EMP_x7,IND_AFFHM,IND_CONS_x7,IND_MAN_x7,IND_WHT_x7,IND_RETT_x7,IND_TWU,IND_INF_x7,IND_FIRR,IND_PSMAW,IND_EHCSA,IND_AERAF,IND_OSER_x7,IND_PUBA_x7,POP_16p_EMP_x8,IND_AFFHM_AFFH,IND_AFFHM_MQE,IND_CONS_x8,IND_MAN_x8,IND_WHT_x8,IND_RETT_x8,IND_TWU_TW,IND_TWU_U,IND_INF_x8,IND_FIRR_FI,IND_FIRR_RR,IND_PSMAW_PST,IND_PSMAW_M,IND_PSMAW_ASWM,IND_EHCSA_E,IND_EHCSA_HCSA,IND_AERAF_AER,IND_AERAF_AF,IND_OSER_x8,IND_PUBA_x8,HU_x14,HU_OCC,HU_VAC,HU_OCC_OWN,HU_OCC_RENT,HU_OCC_OPRlt050,HU_OCC_OPR051100,HU_OCC_OPR101150,HU_OCC_OPR151200,HU_OCC_OPRgt200,POP_16p_EMP_x10,OCC_MBSA,OCC_SERV,OCC_SALES,OCC_NRCM,OCC_PTMM,POP_16p_EMP_x11,OCC_MBSA_MBF,OCC_MBSA_CES,OCC_MBSA_ELCAM,OCC_MBSA_HCPT,OCC_SERV_HCS_x11,OCC_SERV_PS,OCC_SERV_FPS_x11,OCC_SERV_BGM_x11,OCC_SERV_PCS_x11,OCC_SALES_SR_x11,OCC_SALES_OAS_x11,OCC_NRCM_FFF_x11,OCC_NRCM_CE_x11,OCC_NRCM_IMR_x11,OCC_PTMM_P_x11,OCC_PTMM_T_x11,OCC_PTMM_MM_x11,POP_16p_EMP,OCC_MBSA_MBF_M,OCC_MBSA_MBF_BF,OCC_MBSA_CES_CM,OCC_MBSA_CES_AE,OCC_MBSA_CES_LPSS,OCC_MBSA_ELCAM_CSS,OCC_MBSA_ELCAM_L,OCC_MBSA_ELCAM_EL,OCC_MBSA_ELCAM_ADESM,OCC_MBSA_HCPT_HDTP,OCC_MBSA_HCPT_HTT,OCC_SERV_HCS_x12,OCC_SERV_PS_FP,OCC_SERV_PS_LE,OCC_SERV_FPS_x12,OCC_SERV_BGM_x12,OCC_SERV_PCS_x12,OCC_SALES_SR_x12,OCC_SALES_OAS_x12,OCC_NRCM_FFF_x12,OCC_NRCM_CE_x12,OCC_NRCM_IMR_x12,OCC_PTMM_P_x12,OCC_PTMM_T_x12,OCC_PTMM_MM_x12,POP_x13,POP_WHITENH,POP_BLACKNH,POP_AIANNH,POP_ASIANNH,POP_NHPINH,POP_OTH2PLNH,POP_HISP_x13,POP_x14,POP_WHITE,POP_BLACK,POP_AIAN,POP_ASIAN,POP_NHPI,POP_OTH2PL,HU_x15,HU_UIS01D,HU_UIS01A,HU_UIS02,HU_UIS0304,HU_UIS0509,HU_UIS1019,HU_UIS2049,HU_UIS50P,HU_UISOTHER
0,20200620,37,37001,169185,10367022,0.016320,122.952380,104.0,35.0,3.0,2.006526,1.697232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,169185,80687,88498,9733,10441,10964,6617,6695,2813,2547,5816,10436,10507,10160,9534,10761,11540,12198,4627,5579,3885,5390,7165,4477,3800,3500,113559,13953,57362,12324,20999,6608,1009,1304,169185,147007,22178,41495,34829,64210,27670,39447,28503,55151,27409,66311,19741,22838,10741,8117,2989,1395,490,55078,66311,3948,3012,3759,3359,3891,3528,2804,3127,3072,5085,6324,8761,6165,3767,3363,2346,79356,505,6093,12134,2436,10426,3573,760,3841,6894,20057,6294,3229,3114,79356,474,31,6093,12134,2436,10426,2996,577,760,2807,1034,3367,9,3518,7673,12384,1082,5212,3229,3114,72651,66311,6340,44128,22183,48899,16244,1002,152,14,79356,26705,12805,18298,7570,13978,79356,10554,3859,7170,5122,1965,1677,4301,3251,1611,8375,9923,213,4224,3133,7670,2548,3760,79356,7058,3496,1717,1287,855,1312,234,4484,1140,2635,2487,1965,515,1162,4301,3251,1611,8375,9923,213,4224,3133,7670,2548,3760,169185,105121,33083,291,2811,46,5655,22178,169185,114027,33392,580,2898,46,18242,72651,48324,2545,1702,2606,3574,2523,2113,836,8428
1,20200627,37,37001,169185,10367022,0.016320,100.308655,95.0,36.0,1.0,1.636991,1.550356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,169185,80687,88498,9733,10441,10964,6617,6695,2813,2547,5816,10436,10507,10160,9534,10761,11540,12198,4627,5579,3

In [16]:
# # SAVE CSV!
# dt = datetime.today().strftime('%Y%m%d-%H%M%S')

# fpath = os.path.join(DATA_DIR,'processed','scratch',f'cfa_df_{dt}.csv')
# df.to_csv(fpath,index=False)
# print(dt)

# READ BACK AND CHECK
ts = '20241212-050621'
fpath = os.path.join(DATA_DIR,'processed','scratch',f'cfa_df_{ts}.csv')

dff = pd.read_csv(fpath)
# display(df, dff)

In [17]:
dff

,Date,State_fips,Fips,Pop,State_pop,Ratio,Proj_state_inc_deaths,True_state_inc_deaths,True_county_cum_deaths,True_county_inc_deaths,Naive_proj_deaths,Naive_true_deaths,True_county_inc_deaths_tm_4,True_county_inc_deaths_tm_5,True_county_inc_deaths_tm_6,True_county_inc_deaths_tm_7,True_county_inc_deaths_tm_8,True_county_inc_deaths_tm_9,True_county_inc_deaths_tm_10,True_county_inc_deaths_tm_11,POP_x2,POP_M,POP_F,POP_A0004,POP_A0509,POP_A1014,POP_A1517,POP_A1819,POP_A20,POP_A21,POP_A2224,POP_A2529,POP_A3034,POP_A3539,POP_A4044,POP_A4549,POP_A5054,POP_A5559,POP_A6061,POP_A6264,POP_A6566,POP_A6769,POP_A7074,POP_A7579,POP_A8084,POP_A85p,POP_A25p,EDU_ltHS,EDU_HS,EDU_ASSOC,EDU_BACH,EDU_MAST,EDU_PROF,EDU_DOCT,POP_x3,POP_NH,POP_HISP_x12,POP_A0018,POP_A1934,POP_A3564,POP_A65p,HINS_A0018,HINS_A1934,HINS_A3564,HINS_A65p,HH_x5,HH_1P,HH_2P,HH_3P,HH_4P,HH_5P,HH_6P,HH_7pP,MHI,HH_x6,HHI_00_10k,HHI_10_15k,HHI_15_20k,HHI_20_25k,HHI_25_30k,HHI_30_35k,HHI_35_40k,HHI_40_45k,HHI_45_50k,HHI_50_60k,HHI_60_75k,HHI_75_100k,HHI_100_125k,HHI_125_150k,HHI_150_200k,HHI_200k_p,POP_16p_EMP_x7,IND_AFFHM,IND_CONS_x7,IND_MAN_x7,IND_WHT_x7,IND_RETT_x7,IND_TWU,IND_INF_x7,IND_FIRR,IND_PSMAW,IND_EHCSA,IND_AERAF,IND_OSER_x7,IND_PUBA_x7,POP_16p_EMP_x8,IND_AFFHM_AFFH,IND_AFFHM_MQE,IND_CONS_x8,IND_MAN_x8,IND_WHT_x8,IND_RETT_x8,IND_TWU_TW,IND_TWU_U,IND_INF_x8,IND_FIRR_FI,IND_FIRR_RR,IND_PSMAW_PST,IND_PSMAW_M,IND_PSMAW_ASWM,IND_EHCSA_E,IND_EHCSA_HCSA,IND_AERAF_AER,IND_AERAF_AF,IND_OSER_x8,IND_PUBA_x8,HU_x14,HU_OCC,HU_VAC,HU_OCC_OWN,HU_OCC_RENT,HU_OCC_OPRlt050,HU_OCC_OPR051100,HU_OCC_OPR101150,HU_OCC_OPR151200,HU_OCC_OPRgt200,POP_16p_EMP_x10,OCC_MBSA,OCC_SERV,OCC_SALES,OCC_NRCM,OCC_PTMM,POP_16p_EMP_x11,OCC_MBSA_MBF,OCC_MBSA_CES,OCC_MBSA_ELCAM,OCC_MBSA_HCPT,OCC_SERV_HCS_x11,OCC_SERV_PS,OCC_SERV_FPS_x11,OCC_SERV_BGM_x11,OCC_SERV_PCS_x11,OCC_SALES_SR_x11,OCC_SALES_OAS_x11,OCC_NRCM_FFF_x11,OCC_NRCM_CE_x11,OCC_NRCM_IMR_x11,OCC_PTMM_P_x11,OCC_PTMM_T_x11,OCC_PTMM_MM_x11,POP_16p_EMP,OCC_MBSA_MBF_M,OCC_MBSA_MBF_BF,OCC_MBSA_CES_CM,OCC_MBSA_CES_AE,OCC_MBSA_CES_LPSS,OCC_MBSA_ELCAM_CSS,OCC_MBSA_ELCAM_L,OCC_MBSA_ELCAM_EL,OCC_MBSA_ELCAM_ADESM,OCC_MBSA_HCPT_HDTP,OCC_MBSA_HCPT_HTT,OCC_SERV_HCS_x12,OCC_SERV_PS_FP,OCC_SERV_PS_LE,OCC_SERV_FPS_x12,OCC_SERV_BGM_x12,OCC_SERV_PCS_x12,OCC_SALES_SR_x12,OCC_SALES_OAS_x12,OCC_NRCM_FFF_x12,OCC_NRCM_CE_x12,OCC_NRCM_IMR_x12,OCC_PTMM_P_x12,OCC_PTMM_T_x12,OCC_PTMM_MM_x12,POP_x13,POP_WHITENH,POP_BLACKNH,POP_AIANNH,POP_ASIANNH,POP_NHPINH,POP_OTH2PLNH,POP_HISP_x13,POP_x14,POP_WHITE,POP_BLACK,POP_AIAN,POP_ASIAN,POP_NHPI,POP_OTH2PL,HU_x15,HU_UIS01D,HU_UIS01A,HU_UIS02,HU_UIS0304,HU_UIS0509,HU_UIS1019,HU_UIS2049,HU_UIS50P,HU_UISOTHER
0,20200620,37,37001,169185,10367022,0.016320,122.952380,104.0,35.0,3.0,2.006526,1.697232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,169185,80687,88498,9733,10441,10964,6617,6695,2813,2547,5816,10436,10507,10160,9534,10761,11540,12198,4627,5579,3885,5390,7165,4477,3800,3500,113559,13953,57362,12324,20999,6608,1009,1304,169185,147007,22178,41495,34829,64210,27670,39447,28503,55151,27409,66311,19741,22838,10741,8117,2989,1395,490,55078,66311,3948,3012,3759,3359,3891,3528,2804,3127,3072,5085,6324,8761,6165,3767,3363,2346,79356,505,6093,12134,2436,10426,3573,760,3841,6894,20057,6294,3229,3114,79356,474,31,6093,12134,2436,10426,2996,577,760,2807,1034,3367,9,3518,7673,12384,1082,5212,3229,3114,72651,66311,6340,44128,22183,48899,16244,1002,152,14,79356,26705,12805,18298,7570,13978,79356,10554,3859,7170,5122,1965,1677,4301,3251,1611,8375,9923,213,4224,3133,7670,2548,3760,79356,7058,3496,1717,1287,855,1312,234,4484,1140,2635,2487,1965,515,1162,4301,3251,1611,8375,9923,213,4224,3133,7670,2548,3760,169185,105121,33083,291,2811,46,5655,22178,169185,114027,33392,580,2898,46,18242,72651,48324,2545,1702,2606,3574,2523,2113,836,8428
1,20200627,37,37001,169185,10367022,0.016320,100.308655,95.0,36.0,1.0,1.636991,1.550356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,169185,80687,88498,9733,10441,10964,6617,6695,2813,2547,5816,10436,10507,10160,9534,10761,11540,12198,4627,5579,3